In [ ]:
import pandas as pd

# 데이터 로드 (예시)
df = pd.read_csv('factory_data.csv')

# 1. 서울에서 공장이 가장 많은 구는 어디야?
구별_공장수 = df.groupby('구')['공장관리번호'].count().sort_values(ascending=False)
print(f"공장이 가장 많은 구: {구별_공장수.index[0]} ({구별_공장수.iloc[0]}개)")

# 2. 강동구에 있는 공장 중 '임대' 공장은 얼마나 돼?
강동구_임대 = df[(df['구'] == '강동구') & (df['소유구분'] == '임대')]
print(f"강동구 임대 공장 수: {len(강동구_임대)}개")

# 3. 최근 5년간 서울 전체 공장 등록 건수 추이를 보여줘
df['등록년도'] = pd.to_datetime(df['등록일자']).dt.year
최근5년 = df['등록년도'].max() - 4
연도별_등록수 = df[df['등록년도'] >= 최근5년].groupby('등록년도')['공장관리번호'].count()
print("최근 5년 등록 건수 추이:")
print(연도별_등록수)

# 4. 지난 20년간 서울에서 공장 수가 가장 많이 증가한 5개 구를 알려줘
최근20년 = df['등록년도'].max() - 19
df_20년전 = df[df['등록년도'] < 최근20년].groupby('구')['공장관리번호'].count()
df_최근 = df[df['등록년도'] >= 최근20년].groupby('구')['공장관리번호'].count()
증가량 = (df_최근 - df_20년전.reindex(df_최근.index, fill_value=0)).sort_values(ascending=False).head(5)
print("공장 수 증가 상위 5개 구:")
print(증가량)

# 5. 새로 생긴 공장들이 요즘 어느 지역에 많이 몰려 있어?
최근3년 = df['등록년도'].max() - 2
최근_공장 = df[df['등록년도'] >= 최근3년].groupby('구')['공장관리번호'].count().sort_values(ascending=False).head(5)
print("최근 3년 신규 공장이 많은 지역:")
print(최근_공장)

# 6. 어느 구가 제조업 중심지처럼 보이는지 알 수 있을까?
구별_분석 = df.groupby('구').agg({
    '공장관리번호': 'count',
    '직원수': 'sum',
    '공장면적': 'sum'
}).rename(columns={'공장관리번호': '공장수'})
구별_분석['종합점수'] = (구별_분석['공장수'] * 0.4 + 
                        구별_분석['직원수'] * 0.3 + 
                        구별_분석['공장면적'] * 0.3)
제조업_중심지 = 구별_분석.sort_values('종합점수', ascending=False).head(5)
print("제조업 중심지로 보이는 구:")
print(제조업_중심지)

# 7. 공장이지만 주거지와 가까워서 문제가 될 만한 곳이 있는지 찾을 수 있어?
# (주거지역 정보가 있다고 가정)
주거지근접_공장 = df[(df['주거지역거리'] < 500) & (df['업종'].isin(['화학', '금속', '섬유']))]
print(f"주거지 근접 우려 공장: {len(주거지근접_공장)}개")
print(주거지근접_공장.groupby('구')['공장관리번호'].count())

# 8. 여성 직원 비율이 높은 공장은 어떤 업종이 많아?
df['여성비율'] = (df['여성직원수'] / df['직원수']) * 100
여성비율_높은_공장 = df[df['여성비율'] >= 50]
업종별_여성비율 = 여성비율_높은_공장.groupby('업종')['공장관리번호'].count().sort_values(ascending=False)
print("여성 직원 비율이 높은 업종:")
print(업종별_여성비율.head(10))

# 9. 공장 면적 대비 직원 수가 많은 공장과 적은 공장 상위 5개씩 알려줘
df['면적당직원수'] = df['직원수'] / df['공장면적']
많은_공장 = df.nlargest(5, '면적당직원수')[['공장명', '구', '면적당직원수', '직원수', '공장면적']]
적은_공장 = df.nsmallest(5, '면적당직원수')[['공장명', '구', '면적당직원수', '직원수', '공장면적']]
print("면적 대비 직원 수가 많은 공장 TOP 5:")
print(많은_공장)
print("\n면적 대비 직원 수가 적은 공장 TOP 5:")
print(적은_공장)

# 10. 생산품 기준으로 규모가 큰 공장들이 어떤 제품을 만드는지 궁금해
생산품별_규모 = df.groupby('생산품').agg({
    '공장관리번호': 'count',
    '공장면적': 'sum',
    '직원수': 'sum'
}).rename(columns={'공장관리번호': '공장수'})
생산품별_규모 = 생산품별_규모.sort_values('공장면적', ascending=False).head(10)
print("규모가 큰 생산품 종류:")
print(생산품별_규모)

# 11. 외국인 근로자가 많은 공장은 어디야? 업종 특징도 알려줘
df['외국인비율'] = (df['외국인근로자수'] / df['직원수']) * 100
외국인_많은_공장 = df.nlargest(20, '외국인근로자수')
print("외국인 근로자가 많은 공장:")
print(외국인_많은_공장[['공장명', '구', '업종', '외국인근로자수', '외국인비율']])
print("\n외국인 근로자가 많은 업종:")
print(df.groupby('업종')['외국인근로자수'].sum().sort_values(ascending=False).head(10))

In [1]:
import sys
from pathlib import Path

# sub-files 경로를 파이썬 모듈 검색 경로에 추가
BASE_DIR = Path().resolve()  # 현재 노트북이 있는 code 디렉터리
SUB_FILES_DIR = BASE_DIR / "sub-files"
if str(SUB_FILES_DIR) not in sys.path:
    sys.path.append(str(SUB_FILES_DIR))

# 분리한 모듈들 import
from config import df, model
from state import GraphState, generate_session_id
from workflow import graph


def run_query(question: str, context: str | None = None, relevance: str | None = None):
    """LangGraph 파이프라인을 한 번에 실행하는 헬퍼 함수"""
    session_id = generate_session_id()
    state: GraphState = {
        "question": question,
        "q_type": "",
        "answer": "",
        "session_id": session_id,
        "context": context,
        "relevance": relevance,
        "execution_id": None,
    }

    # LangGraph는 thread_id를 필요로 하므로 session_id를 thread_id로도 사용
    result = graph.invoke(
        state, 
        config={"configurable": {"thread_id": session_id, "session_id": session_id}}
    )
    return result


/Users/jeongyunl/Documents/GitHub/factory-chatbot-demo/code/sub-files/config.py:25: DtypeWarning: Columns (11,13,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/cleaned_전국공장등록현황_preprocessed_seoul.csv")


In [2]:
# 예시: 한 번 실행해 보기
example_result = run_query("서울에서 공장이 가장 많은 구는 어디야?")
print("답변:", example_result["answer"])
print("\n전체 결과:", example_result)

🆕 새로운 세션 히스토리 생성: 875982ad...
🆕 새로운 세션 히스토리 생성: aeb7718a...


Error in RootListenersTracer.on_chain_end callback: ValueError("Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got {'question': '서울에서 공장이 가장 많은 구는 어디야?'}.")


답변: [DATA]제공된 데이터에 따르면, 서울특별시에서 공장이 가장 많은 구는 **금천구**이며,  
해당 구에 등록된 공장 수는 **4,059개**입니다.

- 분석 방식:  
  - 전체 데이터 중 `정제_시도명`이 **'서울특별시'**인 행만 필터링  
  - 그 후 `정제_시군구명`(구 단위)별로 `공장관리번호`를 기준으로 공장 수를 집계  
  - 집계된 구들 중 **공장 수가 최댓값인 구**를 추출  
- 그 결과, **금천구가 4,059개로 서울에서 공장이 가장 많은 구**로 나타났습니다.

전체 결과: {'question': '서울에서 공장이 가장 많은 구는 어디야?', 'q_type': 'domain_specific', 'answer': "[DATA]제공된 데이터에 따르면, 서울특별시에서 공장이 가장 많은 구는 **금천구**이며,  \n해당 구에 등록된 공장 수는 **4,059개**입니다.\n\n- 분석 방식:  \n  - 전체 데이터 중 `정제_시도명`이 **'서울특별시'**인 행만 필터링  \n  - 그 후 `정제_시군구명`(구 단위)별로 `공장관리번호`를 기준으로 공장 수를 집계  \n  - 집계된 구들 중 **공장 수가 최댓값인 구**를 추출  \n- 그 결과, **금천구가 4,059개로 서울에서 공장이 가장 많은 구**로 나타났습니다.", 'session_id': '875982ad-a8f8-435f-b8ef-b73e8da6e722', 'context': None, 'relevance': None, 'execution_id': None}


In [19]:
import pandas as pd

# 필터 조건 설정
구_filter = '강남구'
업종코드_filter = '14112'

# 강남구 & 해당 업종코드 필터링
filtered = df[
    (df['정제_시군구명'] == 구_filter) &
    (df['정제_대표업종'] == 14112)
]

# 공장관리번호 기준으로 공장 수 집계 (중복 제거)
factory_counts = (
    filtered
    .drop_duplicates(subset=['공장관리번호'])
    .groupby('회사명', as_index=False)['공장관리번호']
    .nunique()
    .rename(columns={'공장관리번호': '공장수'})
)

# 공장 수 기준 내림차순 정렬 후 최다 공장 보유 회사 추출
max_factory_count = factory_counts['공장수'].max() if not factory_counts.empty else 0

result = factory_counts[factory_counts['공장수'] == max_factory_count] if max_factory_count > 0 else factory_counts

# 최종 결과를 return_var에 할당
return_var = result
result

,회사명,공장수
0,(주)규성,1
1,(주)데무,1
2,(주)마당,1
3,(주)미스지콜렉션,1
4,(주)사라,1
5,(주)신형물산,1
6,(주)이상봉,1
7,(주)진세공영,1
8,(주)진태옥,1
9,(주)케이씨에스 스포츠,1


In [18]:
df[
    (df['정제_시군구명'] == 구_filter)
]['정제_대표업종'].unique()


array([18111, 33932, 14191, 28123, 18119, 61299, 14112, 26321, 26410,
       10220, 26329, 26299, 26421, 10412, 59201, 26293, 29280, 27195,
       28901, 17902, 26310, 27309, 27216, 14111, 28422, 10519, 33910,
       26429, 25112, 27192, 10891, 13221, 29172, 28519, 33999, 29299,
       13223, 10602, 27199, 14130, 13222, 29176, 31312, 21100, 21301,
       23222, 29120, 14192, 27215, 27214, 14300, 58113, 28909, 26323,
       33120, 20423, 33110, 18112, 27211, 26521, 27220, 27212, 26519,
       32091, 28423, 18200, 29175, 23322, 20421, 20202, 28903, 26111,
       14200, 28119, 26529])